<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [15]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from functions import util
import json
import os
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [16]:
# If you haven't set the env variable 'HOPSWORKS_API_KEY', then uncomment the next line and enter your API key
# os.environ["HOPSWORKS_API_KEY"] = ""
import os
import dotenv
dotenv.load_dotenv()

api_key = os.environ.get("HOPSWORKS_API_KEY")
os.environ["HOPSWORKS_API_KEY"] = api_key

project = hopsworks.login()
fs = project.get_feature_store() 
secrets = util.secrets_api(project.name)

# This line will fail if you have not registered the AQI_API_KEY as a secret in Hopsworks
AQI_API_KEY = secrets.get_secret("AQI_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()

location_str

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1161368
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


'{"country": "vietnam", "city": "hanoi", "street": "langha", "aqicn_url": "https://api.waqi.info/feed/@8641", "latitude": 21.03, "longitude": 105.85}'

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [24]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [ ]:
# Get today's date
today = datetime.date.today()

previous_7_day = (today - datetime.timedelta(days=7)).strftime("%Y-%m-%d 00:00:00")

# Fill today's AQ with lagged PM25 from previous 1,2,3 days
query = air_quality_fg.select_all()
query.filter(air_quality_fg.date >= previous_7_day)

previous_7_day = query.read()
previous_7_day = previous_7_day.sort_values(by='date', ascending=False)

# take the 1st 3 rows
previous_3_days = previous_7_day.head(3)
previous_3_days


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.99s) 


,date,pm25,pm25_lag1,pm25_lag2,pm25_lag3,country,city,street,url
4,2024-11-12 00:00:00+00:00,160.0,172.0,159.0,166.0,vietnam,hanoi,langha,https://api.waqi.info/feed/@8641
0,2024-11-09 00:00:00+00:00,172.0,159.0,166.0,162.0,vietnam,hanoi,langha,https://api.waqi.info/feed/@8641
1,2024-11-08 00:00:00+00:00,159.0,166.0,162.0,149.0,vietnam,hanoi,langha,https://api.waqi.info/feed/@8641


In [21]:
aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQI_API_KEY)

# adding the lagged PM25 values from previous_3_days
aq_today_df['pm25_lag1'] = previous_3_days.iloc[0]['pm25']
aq_today_df['pm25_lag2'] = previous_3_days.iloc[1]['pm25']
aq_today_df['pm25_lag3'] = previous_3_days.iloc[2]['pm25']

aq_today_df


,pm25,country,city,street,date,url,pm25_lag1,pm25_lag2,pm25_lag3
0,159.0,vietnam,hanoi,langha,2024-11-13,https://api.waqi.info/feed/@8641,160.0,172.0,159.0


In [ ]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   pm25       1 non-null      float32       
 1   country    1 non-null      object        
 2   city       1 non-null      object        
 3   street     1 non-null      object        
 4   date       1 non-null      datetime64[ns]
 5   url        1 non-null      object        
 6   pm25_lag1  1 non-null      float32       
 7   pm25_lag2  1 non-null      float32       
 8   pm25_lag3  1 non-null      float32       
dtypes: datetime64[ns](1), float32(4), object(4)
memory usage: 188.0+ bytes


## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [ ]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 21.0°N 105.75°E
Elevation 24.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2024-11-13,26.450001,0.0,11.435313,151.821503,hanoi
1,2024-11-14,27.700001,0.0,6.792466,147.994659,hanoi
2,2024-11-15,27.000000,0.0,14.654254,152.175995,hanoi
3,2024-11-16,26.450001,0.0,8.435069,140.194473,hanoi
4,2024-11-17,27.100000,0.0,0.509117,224.999893,hanoi
5,2024-11-18,25.400000,0.0,4.379589,189.462250,hanoi
6,2024-11-19,24.350000,0.0,4.024922,26.564985,hanoi
7,2024-11-20,23.650000,0.0,5.692099,34.695221,hanoi
8,2024-11-21,21.600000,0.0,6.120000,118.072395,hanoi
9,2024-11-22,22.450001,0.0,6.409617,128.157272,hanoi


In [ ]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         10 non-null     datetime64[ns]
 1   temperature_2m_mean          10 non-null     float32       
 2   precipitation_sum            10 non-null     float32       
 3   wind_speed_10m_max           10 non-null     float32       
 4   wind_direction_10m_dominant  10 non-null     float32       
 5   city                         10 non-null     object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 452.0+ bytes


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [ ]:
# Insert new data
weather_fg.insert(daily_df)

2024-11-13 17:29:16,520 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1161368/fs/1152071/fg/1346898


Uploading Dataframe: 0.00% |          | Rows 0/10 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1161368/jobs/named/weather_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x1204172d0>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "wind_speed_10m_max",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 670726
         }
       },
       "result": {
         "observed_value": 0.5091168880462646,
         "element_count": 10,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-13T04:29:16.000518Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_m

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
